#  第12章 高阶pandas

## 本章会介绍pandas 的Categorical 类型。我们将展示在使用pandas进行某种操作时如果获得更好的性能和内存使用。我们还会介绍一些在统计和机器学习应用中使用分类数据的工具。

### 12.1.1 背景和目标

#### 一个列经常会包含重复值，这些重复值是小型的不同值的集合。我们已经看见向unique,value_counts这样的函数，它们允许我们从一个数组中提取不同值并并分别计算这些不同值的频率。 

In [3]:
a=[1,2,3,4,4]

In [4]:
import numpy as np
import pandas as pd

In [5]:
s=pd.unique(a)

In [6]:
type(a)

list

In [7]:
type(s)

numpy.ndarray

In [8]:
values=pd.Series(['apple','orange','apple','apple']*2)

In [9]:
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [10]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [11]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

#### 在数据入库的操作中，使用所谓的维度表是一种最佳的实践，维度表包含了不同值，并将主要观测值存储为引用维度表的整数键 

In [12]:
values=pd.Series([0,1,0,0]*2)

In [13]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [14]:
dim=pd.Series(['apple','orange'])

In [15]:
dim

0     apple
1    orange
dtype: object

#### 我们可以使用take 方法来恢复原来的字符串Series:

In [16]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

#### 这种按照整数展现的方式被称为分类或字典编码展现。不同值的数组可以被称为数据类别，字典或层级。 

### 12.1.2pandas 中的Categorical 类型

####  pandas 拥有特殊的Categorical 类型，用于承载基于整数的类别展示或编码的数据。让我们考虑之前的示例Series:

In [17]:
fruits=['apple','orange','apple','apple']*2

In [18]:
N=len(fruits)

In [19]:
df=pd.DataFrame({'fruit':fruits,'basket_id':np.arange(N),'count':np.random.randint(3,15,size=N),'weight':np.random.uniform(0,4,size=N)},columns=['basket_id','fruit','count','weight'])

In [20]:
df

,basket_id,fruit,count,weight
0,0,apple,9,1.782864
1,1,orange,9,1.631684
2,2,apple,9,2.057082
3,3,apple,9,1.742667
4,4,apple,7,3.574675
5,5,orange,14,0.682741
6,6,apple,9,2.503771
7,7,apple,3,3.574109


#### 这里，df['fruit'] 是一个Python字符串组成的数组。我们可以通过调用函数将它转换为Categorical 对象：

In [21]:
fruit_cat=df['fruit'].astype('category')

In [22]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

In [23]:
type(fruit_cat)

pandas.core.series.Series

In [24]:
type(df['fruit'])

pandas.core.series.Series

In [25]:
c=fruit_cat.values

In [26]:
type(c)

pandas.core.arrays.categorical.Categorical

#### Categorical对象拥有categories和code 属性：

In [27]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [28]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

#### 你可以通过分配已经转换的结果将DataFrame的一列转换为Categorical对象: 

In [29]:
df['fruit']=df['fruit'].astype('category')

In [30]:
df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): [apple, orange]

#### 你也可以从其他python 序列类型直接生成pandas.Categorical: 

In [31]:
my_categories=pd.Categorical(['foo','bar','baz','foo','bar'])

In [32]:
my_categories

[foo, bar, baz, foo, bar]
Categories (3, object): [bar, baz, foo]

#### 如果你已经从另一个数据源获得了分类编码数据，你可以使用from_codes构造函数：

In [33]:
categories=['foo','bar','baz']

In [34]:
codes=[0,1,2,0,0,1]

In [35]:
my_cats_2=pd.Categorical.from_codes(codes,categories)

In [36]:
my_cats_2

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo, bar, baz]

####  除非显示地指定，分类转换是不会指定类别地顺序的。因此categories数组可能会与输入数据的顺序不同。当使用from_codes或其他任意构造函数时，你可以为类别指定一个有意义的顺序：

In [37]:
odered_cat=pd.Categorical.from_codes(codes,categories,ordered=True)

In [38]:
odered_cat

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

In [39]:
my_cats_2.as_ordered() # 一个未排序的分类实例可以使用as_ordered进行排序

[foo, bar, baz, foo, foo, bar]
Categories (3, object): [foo < bar < baz]

#### 最后需要注意，分类数据可以不是字符串，尽管我举的例子都是字符串的例子，一个分类数组可以包含任一不可变的值类型。 

### 12.1.3 使用Categorical 对象进行计算

####  在pandas 中使用Categorical 与非编码版本相比（例如字符串数组）整体上是一致的。pandas中的一部分，比如groupby 函数，在与Categorical对象协同工作时，性能更好。还有一些函数可以利用ordered标识。 

#### 让我们考虑随机数字数据，并使用pandas.qcut 分箱函数。结果会返回pandas.Categorical;本书之前使用过pandas.cut 但之前忽略了分类如何工作的细节

In [40]:
np.random.seed(12345)

In [41]:
draws=np.random.randn(1000)

In [42]:
draws[:5]

array([-0.20470766,  0.47894334, -0.51943872, -0.5557303 ,  1.96578057])

In [43]:
bins=pd.qcut(draws,4)   # 计算四分位分箱，并提取一些统计值

In [44]:
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [45]:
max(draws)

3.92752804079273

#### 虽然样本的四分位数有用，但是在生成一份报告时，四分位数没有四分位名称有用。我们可以通过在qcut函数中使用labels参数来实现这个功能： 

In [46]:
bins=pd.qcut(draws,4,labels=['Q1','Q2','Q3','Q4'])

In [47]:
bins

[Q2, Q3, Q2, Q2, Q4, ..., Q3, Q2, Q1, Q3, Q4]
Length: 1000
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

In [48]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

#### 被标记的bins分类数据并不包含数据中箱体边界的相关信息，因此我们可以使用groupby来提取一些汇总统计值：

In [49]:
bins=pd.Series(bins,name='quartile')

In [50]:
results=(pd.Series(draws).groupby(bins).agg(['count','min','max']))  # 看不懂？reset_index有什么用

In [51]:
results

,count,min,max
quartile,,,
Q1,250,-2.949343,-0.685484
Q2,250,-0.683066,-0.010115
Q3,250,-0.010032,0.628894
Q4,250,0.634238,3.927528


In [52]:
results1=(pd.Series(draws).groupby(bins).agg(['count','min','max']).reset_index())  # 看不懂？reset_index 有什么用

In [53]:
results1

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [54]:
results1.iloc[0]

quartile          Q1
count            250
min         -2.94934
max        -0.685484
Name: 0, dtype: object

In [55]:
results.iloc[:,0]

quartile
Q1    250
Q2    250
Q3    250
Q4    250
Name: count, dtype: int64

####  结果中的'quartile'列保留了bins中原始的分类信息，包括顺序

In [56]:
results1['quartile']

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): [Q1 < Q2 < Q3 < Q4]

#### 12.1.3.1使用分类获得更高性能 

#### 如果你在特定数据集上做了大量的分析，将数据转换为分类数据可以产生大幅的性能提升。DataFrame 中一列的分类版本通常会明显使用更少的内存。让我们考虑一些含有一千万元素的Series以及少量的不同类别： 

In [57]:
N=10000000

In [58]:
draws=pd.Series(np.random.randn(N))

In [59]:
labels=pd.Series(['foo','bar','baz','qux']*(N//4))

#### 现在我们将labels转换为Categorical 对象： 

In [60]:
categories=labels.astype('category')

In [61]:
labels.memory_usage()   #比较两者的内存

80000080

In [62]:
categories.memory_usage()

10000272

#### 当然分类转换不是免费的，但是它是一次性开销：

In [63]:
%time _=labels.astype('category')

Wall time: 1.05 s


### 12.1.4分类方法 

#### Series包含的分类数据拥有一些特殊方法，这些方法类似于Series.str的特殊字符串方法。这些方法提供了快捷访问类别和代码的方式。考虑下面的Series:

In [64]:
s=pd.Series(['a','b','c','d']*2)

In [65]:
type(s)

pandas.core.series.Series

In [66]:
s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: object

In [67]:
cat_s=s.astype('category')

In [68]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): [a, b, c, d]

In [69]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [70]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

#### 假设我们知道该数据的实际类别集合超过了数据中观察到的四个值。 我们可以使用set_categories 方法来改变类别：

In [71]:
actual_categories=['a','b','c','d','e']

In [72]:
cat_s2=cat_s.cat.set_categories(actual_categories)

In [73]:
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): [a, b, c, d, e]

####  虽然看起来数据并未改变，但新类别将反映在使用它们的操作中。例如，value_counts将遵循新的类别（如果存在）：

In [74]:
cat_s.value_counts()

d    2
c    2
b    2
a    2
dtype: int64

In [75]:
cat_s2.value_counts()

d    2
c    2
b    2
a    2
e    0
dtype: int64

#### 在大型数据集中，分类数据经常用于节省内存和更高性能的便捷工具。在你过滤了一个大型DataFrame或Series之后，很多类别将不会出现在数据中。为了帮助解决这个问题，我们可以使用remove_unused_categories 方法来去除未观察到的类别。

In [88]:
cat_s3=cat_s[cat_s.isin(['a','b'])]

In [89]:
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): [a, b, c, d]

In [90]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

In [92]:
cat_s4=cat_s3.cat.remove_unused_categories()

In [93]:
cat_s4

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): [a, b]

#### 12.1.4.1 创建用于建模的虚拟变量 

#### 当你使用统计数据或机器学习工具时，通常将分类数据转换为虚拟变量。这会产生一个DataFrame,每个不同类别都是它的一列。这些列包含一个特定类别的出现次数，否则为0.

In [79]:
cat_s=pd.Series(['a','b','c','d']*2,dtype='category')  #之前在第7章中提过，pandas.get_dummies函数将一维的分类数据转换为一个包含虚拟变量的DataFrame:

In [80]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1


## 12.2高阶GroupBy应用

### 12.2.1分组转换和“展开”GroupBy 

####  在第10章中，我们在分组操作中学习了apply方法，用于执行转换操作，还有另外一个内建方法transform,与apply 方法类似但是会对你可以使用的函数种类加上更多的限制：

In [81]:
df=pd.DataFrame({'key':['a','b','c']*4,'value':np.arange(12.)})

In [82]:
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [83]:
g=df.groupby('key').value

In [84]:
g.mean()

key
a    4.5
b    5.5
c    6.5
Name: value, dtype: float64

 假设我们想要产生一个Series,它的尺寸和df['value']一样，但值都被'key'分组的均值替代。可以向transform传递匿名函数lambda x:x.mean():

In [85]:
g.transform(lambda x:x.mean())

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

对于内建的聚合函数，我们可以像GroupBy 的agg方法一样传递一个字符串别名：

In [86]:
g.transform('mean')

0     4.5
1     5.5
2     6.5
3     4.5
4     5.5
5     6.5
6     4.5
7     5.5
8     6.5
9     4.5
10    5.5
11    6.5
Name: value, dtype: float64

In [87]:
g.apply(mean)    #  apply 怎么算平均值的

NameError: name 'mean' is not defined

与apply 类似，transform 可以与返回Series的函数一起使用，但结果必须和输入有相同的大小。例如，我们可以使用lambda函数给每个组乘以2：

In [ ]:
g.transform(lambda x:x*2)

作为更复杂的例子 ，我们可以按照每个组的降序计算排名：

In [ ]:
g.transform(lambda x:x.rank(ascending=False))

考虑一个由简单聚合构成的分组转换函数：

In [ ]:
def normalize(x):
    return (x - x.mean()) / x.std()

### 12.2.2 分组的时间重新采样 

 对于时间序列，resample方法在语义上是一种基于时间分段的分组操作。下面是一个小的示例表：

In [94]:
N=15

In [ ]:
import pandas as pd
import numpy as np

In [95]:
times=pd.date_range('2017-05-20 00:00',freq='1min',periods=N)

In [96]:
df=pd.DataFrame({'time':times,'value':np.arange(N)})

In [97]:
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


这里我们我们可以按‘time’ 进行索引，然后重新采样：

In [98]:
df.set_index('time').resample('5min').count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


In [99]:
df2=pd.DataFrame({'time':times.repeat(3),'key':np.tile(['a','b','c'],N),'value':np.arange(N*3)})

In [100]:
df2

,time,key,value
0,2017-05-20 00:00:00,a,0
1,2017-05-20 00:00:00,b,1
2,2017-05-20 00:00:00,c,2
3,2017-05-20 00:01:00,a,3
4,2017-05-20 00:01:00,b,4
5,2017-05-20 00:01:00,c,5
6,2017-05-20 00:02:00,a,6
7,2017-05-20 00:02:00,b,7
8,2017-05-20 00:02:00,c,8
9,2017-05-20 00:03:00,a,9


 要为每个“key”的值进行相同的重新采样，我们可以使用pandas.TimeGrouper对象：

In [101]:
time_key=pd.TimeGrouper('5min')

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: pd.TimeGrouper is deprecated and will be removed; Please use pd.Grouper(freq=...)
  """Entry point for launching an IPython kernel.


之后我们可以设置时间索引，按“key”,time_key 进行分组，再聚合：

In [102]:
time_key

TimeGrouper(freq=<5 * Minutes>, axis=0, sort=True, closed='left', label='left', how='mean', convention='e', base=0)

In [104]:
resampled=(df2.set_index('time').groupby(['key',time_key]).sum())

In [105]:
resampled

value
key time                      
a   2017-05-20 00:00:00     30
    2017-05-20 00:05:00    105
    2017-05-20 00:10:00    180
b   2017-05-20 00:00:00     35
    2017-05-20 00:05:00    110
    2017-05-20 00:10:00    185
c   2017-05-20 00:00:00     40
    2017-05-20 00:05:00    115
    2017-05-20 00:10:00    190

In [106]:
resampled.reset_index()

,key,time,value
0,a,2017-05-20 00:00:00,30
1,a,2017-05-20 00:05:00,105
2,a,2017-05-20 00:10:00,180
3,b,2017-05-20 00:00:00,35
4,b,2017-05-20 00:05:00,110
5,b,2017-05-20 00:10:00,185
6,c,2017-05-20 00:00:00,40
7,c,2017-05-20 00:05:00,115
8,c,2017-05-20 00:10:00,190


In [112]:
resampled.reset_index()

,key,time,value
0,a,2017-05-20 00:00:00,30
1,a,2017-05-20 00:05:00,105
2,a,2017-05-20 00:10:00,180
3,b,2017-05-20 00:00:00,35
4,b,2017-05-20 00:05:00,110
5,b,2017-05-20 00:10:00,185
6,c,2017-05-20 00:00:00,40
7,c,2017-05-20 00:05:00,115
8,c,2017-05-20 00:10:00,190


In [113]:
resampled    #使用TimeGrouper的一个限制是时间必须是Series 或DataFrame的索引

value
key time                      
a   2017-05-20 00:00:00     30
    2017-05-20 00:05:00    105
    2017-05-20 00:10:00    180
b   2017-05-20 00:00:00     35
    2017-05-20 00:05:00    110
    2017-05-20 00:10:00    185
c   2017-05-20 00:00:00     40
    2017-05-20 00:05:00    115
    2017-05-20 00:10:00    190

## 12.3方法链技术

 在向数据集应用一系列变换时，你可能会发现自己创建了许多临时变量，而这些变量在分析中从未使用过。例如，考虑以下例子：

In [ ]:
df=load_data()
df2=df[df['col2']<0]
df2['col1_demeaned']=df2['col1']-df['col2'].mean()
result=df2.groupby('key').col1_demeaned.std()

In [115]:
# 尽管我们在这里并未使用真实数据，但是这个例子体现了一些新方法。首先，DataFrame.assign 方法是对df[k]=v 的赋值方式一种功能替代。
#它返回的是一个按指定修改的新的DataFrame,而不是在原对象上进行修改。因此，下面这些表述是等价的：
#常见的非函数方式
df333=df.copy()
df333['k']=v
#函数赋值的方式
df444=df.assign(k=v)

NameError: name 'v' is not defined

In [ ]:
#原位赋值可能比使用assign 更为快捷，但assign 可以实现更方便的方法链：
result=(df2.assign(col1_demeaned+df2.col1-df2.col2.mean()).groupby('key').col1_demeaned.std())

In [ ]:
 #在做方法链时要牢记你可能会需要引用临时对象。在之前的例子中，我们无法引用load_data 的结果，除非它被赋值给临时变量df。为了处理这种情况，assign和很多其他的pandas 函数接受函数型的参数，这种参数也被称为可调用参数。
 #为了展示操作中的可调用对象，考虑下面这段之前讲过的代码段：
df=load_data()
df2=df[df['col']<0
# 上面的代码可以改写为：
       df=(load_data()
          [lambda x: x['col2']<0])
#这里，load_data的结果没有复制给一个变量，因此传递进[]的函数将会被绑定到方法链那一阶段对象上

### 12.3.1 pipe方法 

有时候你需要使用自定义的函数或来自第三方库的函数，这就是pipe 方法出现的原因。

In [ ]:
#考虑下面一个函数调用序列：
a=f(df,arg1=v1)
b=g(a,v2,arg3=v3)
c=h(b,arg4=v4)

#在使用接受并返回Series 或DataFrame 对象的函数时，你可以调用pipe 方法重写代码：
result=(df.pipe(f,arg1=v1).pipe(g,v2,arg3=v3).pipe(h,arg4=v4))
#表达式f(df)和df.pipe(f)是等价的，但是pipe使得链式调用更为方便。

In [117]:
#将操作的序列泛化成可复用的函数是pipe 方法的一个潜在用途。作为示例，让我们考虑从一列中减去分组平均值：
g=df.groupby(['key1','key2'])
df['col1']=df['col1']-g.transform('mean')
#假设你想要对多列去除均值并方便地改变分组建。此外，你可能想要将转换在方法链中执行。下面是yige 示例实现：
def group_demean(df,by,cols):
    result=df.copy()
    g=df.groupby(by)
    for c in cols:
        result[c]=df[c]-g[c].transform('mean')
    return result
# 之后可以写如下代码：
result=(df[df.col1<0].pipe(group_demean,['key1','key2'],['col1']))

KeyError: 'key1'